In [1]:
!pip install scikit-optimize

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import joblib as jb
import numpy as np
import pandas as pd
import os
import sys
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
import skopt
from sklearn.pipeline import Pipeline
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from catboost import CatBoostClassifier
import catboost
import sklearn
from joblib import Parallel, delayed

# Package versions
print("Python version:", sys.version)
print("Joblib version:", jb.__version__)
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("scikit-optimize version:", skopt.__version__)
print("scikit-learn version:", sklearn.__version__)
print("catboost version:", catboost.__version__)

Python version: 3.10.12 (main, Feb  4 2025, 14:57:36) [GCC 11.4.0]
Joblib version: 1.4.2
NumPy version: 1.26.4
Pandas version: 2.2.3
scikit-optimize version: 0.10.2
scikit-learn version: 1.4.1.post1
catboost version: 1.2.7


In [3]:
data_per_year = jb.load("data_per_year.pkl")
data_per_year[2015].head()

,sex,healthcare_plan,procedure,age,no_show,distance_to_clinic,waiting_time,day_of_week,month,hour,is_holiday_tomorrow,was_holiday_yesterday,no_show_history,no_show_rate_last_1_day_in_MR,median_no_show_rate_last_7_days_in_MR,median_no_show_rate_last_30_days_in_MR,quantile25_no_show_rate_last_7_days_in_MR,quantile25_no_show_rate_last_30_days_in_MR,quantile75_no_show_rate_last_7_days_in_MR,quantile75_no_show_rate_last_30_days_in_MR,mean_no_show_rate_last_7_days_in_MR,mean_no_show_rate_last_30_days_in_MR,min_no_show_rate_last_7_days_in_MR,min_no_show_rate_last_30_days_in_MR,max_no_show_rate_last_7_days_in_MR,max_no_show_rate_last_30_days_in_MR,std_no_show_rate_last_7_days_in_MR,std_no_show_rate_last_30_days_in_MR,no_show_rate_last_1_day_clinic_wide,median_no_show_rate_last_7_days_clinic_wide,median_no_show_rate_last_30_days_clinic_wide,quantile25_no_show_rate_last_7_days_clinic_wide,quantile25_no_show_rate_last_30_days_clinic_wide,quantile75_no_show_rate_last_7_days_clinic_wide,quantile75_no_show_rate_last_30_days_clinic_wide,mean_no_show_rate_last_7_days_clinic_wide,mean_no_show_rate_last_30_days_clinic_wide,min_no_show_rate_last_7_days_clinic_wide,min_no_show_rate_last_30_days_clinic_wide,max_no_show_rate_last_7_days_clinic_wide,max_no_show_rate_last_30_days_clinic_wide,std_no_show_rate_last_7_days_clinic_wide,std_no_show_rate_last_30_days_clinic_wide,today_demand_MR,demand_last_1_day_MR,median_demand_last_7_days_MR,median_demand_last_30_days_MR,quantile25_demand_last_7_days_MR,quantile25_demand_last_30_days_MR,quantile75_demand_last_7_days_MR,quantile75_demand_last_30_days_MR,mean_demand_last_7_days_MR,mean_demand_last_30_days_MR,min_demand_last_7_days_MR,min_demand_last_30_days_MR,max_demand_last_7_days_MR,max_demand_last_30_days_MR,std_demand_last_7_days_MR,std_demand_last_30_days_MR,today_demand_clinic_wide,demand_last_1_day_clinic_wide,median_demand_last_7_days_clinic_wide,median_demand_last_30_days_clinic_wide,quantile25_demand_last_7_days_clinic_wide,quantile25_demand_last_30_days_clinic_wide,quantile75_demand_last_7_days_clinic_wide,quantile75_demand_last_30_days_clinic_wide,mean_demand_last_7_days_clinic_wide,mean_demand_last_30_days_clinic_wide,min_demand_last_7_days_clinic_wide,min_demand_last_30_days_clinic_wide,max_demand_last_7_days_clinic_wide,max_demand_last_30_days_clinic_wide,std_demand_last_7_days_clinic_wide,std_demand_last_30_days_clinic_wide
0,F,Ipasgo,Cervical Spine MRI,69.0,0.0,4.668,15.0,4,1,9,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.046875,0.095238,0.060628,0.0,0.0,0.666667,0.666667,0.233285,0.143218,0.222222,0.178082,0.127717,0.147606,0.089112,0.236111,0.176419,0.191913,0.135563,0.1,0.019231,0.297872,0.297872,0.064484,0.067978,11.0,2.0,3.0,8.0,2.0,4.25,7.0,11.0,4.714286,7.7,1.0,1.0,11.0,18.0,3.76883,4.510358,30.0,18.0,30.0,73.5,23.0,34.25,47.0,85.75,37.0,64.466667,16.0,16.0,73.0,109.0,18.624101,28.996245
1,F,Ipasgo,Knee MRI,50.0,0.0,NaN,18.0,4,1,19,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.046875,0.095238,0.060628,0.0,0.0,0.666667,0.666667,0.233285,0.143218,0.222222,0.178082,0.127717,0.147606,0.089112,0.236111,0.176419,0.191913,0.135563,0.1,0.019231,0.297872,0.297872,0.064484,0.067978,11.0,2.0,3.0,8.0,2.0,4.25,7.0,11.0,4.714286,7.7,1.0,1.0,11.0,18.0,3.76883,4.510358,30.0,18.0,30.0,73.5,23.0,34.25,47.0,85.75,37.0,64.466667,16.0,16.0,73.0,109.0,18.624101,28.996245
2,F,Hapvida,Knee MRI,39.0,0.0,13.090,18.0,4,1,19,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.046875,0.095238,0.060628,0.0,0.0,0.666667,0.666667,0.233285,0.143218,0.222222,0.178082,0.127717,0.147606,0.089112,0.236111,0.176419,0.191913,0.135563,0.1,0.019231,0.297872,0.297872,0.064484,0.067978,11.0,2.0,3.0,8.0,2.0,4.25,7.0,11.0,4.714286,7.7,1.0,1.0,11.0,18.0,3.76883,4.510358,30.0,18.0,30.0,73.5,23.0,34.25,47.0,85.75,37.0,64.466667,16.0,16.0,73.0,109.0,18.624101,28.996245
3,F,Ipasgo,Knee MRI,17.0,0.0,75.092,18.0,4,1,19,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.046875,0.095238,0.060628,0.0,0.0,0.666667,0.666667,0.233285,0.143218,0.222222,0.178082,0.127717,0.147606,0

In [4]:
# Preprocessing step for historic data from the clinic
# 1) No-show statistics
no_show_rate_cols = ['no_show_rate_last_1_day_in_MR',
       'median_no_show_rate_last_7_days_in_MR',
       'median_no_show_rate_last_30_days_in_MR',
       'quantile25_no_show_rate_last_7_days_in_MR',
       'quantile25_no_show_rate_last_30_days_in_MR',
       'quantile75_no_show_rate_last_7_days_in_MR',
       'quantile75_no_show_rate_last_30_days_in_MR',
       'mean_no_show_rate_last_7_days_in_MR',
       'mean_no_show_rate_last_30_days_in_MR',
       'min_no_show_rate_last_7_days_in_MR',
       'min_no_show_rate_last_30_days_in_MR',
       'max_no_show_rate_last_7_days_in_MR',
       'max_no_show_rate_last_30_days_in_MR',
       'std_no_show_rate_last_7_days_in_MR',
       'std_no_show_rate_last_30_days_in_MR',
       'no_show_rate_last_1_day_clinic_wide',
       'median_no_show_rate_last_7_days_clinic_wide',
       'median_no_show_rate_last_30_days_clinic_wide',
       'quantile25_no_show_rate_last_7_days_clinic_wide',
       'quantile25_no_show_rate_last_30_days_clinic_wide',
       'quantile75_no_show_rate_last_7_days_clinic_wide',
       'quantile75_no_show_rate_last_30_days_clinic_wide',
       'mean_no_show_rate_last_7_days_clinic_wide',
       'mean_no_show_rate_last_30_days_clinic_wide',
       'min_no_show_rate_last_7_days_clinic_wide',
       'min_no_show_rate_last_30_days_clinic_wide',
       'max_no_show_rate_last_7_days_clinic_wide',
       'max_no_show_rate_last_30_days_clinic_wide',
       'std_no_show_rate_last_7_days_clinic_wide',
       'std_no_show_rate_last_30_days_clinic_wide']

# 2) Demand statistics
demand_cols = ['demand_last_1_day_MR', 'median_demand_last_7_days_MR',
       'median_demand_last_30_days_MR', 'quantile25_demand_last_7_days_MR',
       'quantile25_demand_last_30_days_MR', 'quantile75_demand_last_7_days_MR',
       'quantile75_demand_last_30_days_MR', 'mean_demand_last_7_days_MR',
       'mean_demand_last_30_days_MR', 'min_demand_last_7_days_MR',
       'min_demand_last_30_days_MR', 'max_demand_last_7_days_MR',
       'max_demand_last_30_days_MR', 'std_demand_last_7_days_MR',
       'std_demand_last_30_days_MR', 'demand_last_1_day_clinic_wide',
       'median_demand_last_7_days_clinic_wide',
       'median_demand_last_30_days_clinic_wide',
       'quantile25_demand_last_7_days_clinic_wide',
       'quantile25_demand_last_30_days_clinic_wide',
       'quantile75_demand_last_7_days_clinic_wide',
       'quantile75_demand_last_30_days_clinic_wide',
       'mean_demand_last_7_days_clinic_wide',
       'mean_demand_last_30_days_clinic_wide',
       'min_demand_last_7_days_clinic_wide',
       'min_demand_last_30_days_clinic_wide',
       'max_demand_last_7_days_clinic_wide',
       'max_demand_last_30_days_clinic_wide',
       'std_demand_last_7_days_clinic_wide',
       'std_demand_last_30_days_clinic_wide']

# Corresponding PCA objects
pca_no_show = PCA(whiten=True, n_components=0.95)
pca_demand  = PCA(whiten=True, n_components=0.95)

# Preparing transformers list
transformers = [
    ("sex", SimpleImputer(strategy="most_frequent"), ["sex"]), # Just need to impute missing values
    ("PCA_no_show", pca_no_show, no_show_rate_cols),
    ("PCA_demand", pca_demand, demand_cols),
    ("numerical_no_indicator", Pipeline([
        ("standardize", StandardScaler()),
        ("impute", SimpleImputer(strategy="median")) # simple_imputer without missing indicator
    ]), ["age", "distance_to_clinic"]),
    ("numerical", Pipeline([
        ("standardize", StandardScaler()),
        ("impute", SimpleImputer(strategy="median", add_indicator=True)) # simple_imputer with missing indicator
    ]), ["waiting_time", "no_show_history", "today_demand_clinic_wide", "today_demand_MR"])
]

# Initializing ColumnTransformer
col_transformer = ColumnTransformer(transformers, remainder="passthrough")

# Wrapper to automatically create DataFrames after transforming data
class DFConverter(BaseEstimator, TransformerMixin):
    def __init__(self, transformer, cat_cols):
        self.transformer = transformer
        self.cat_cols = cat_cols

    def fit(self, X, y=None):
        self.transformer.fit(X, y)
        # Get feature names
        self.feature_names_ = list(map(lambda c: c.split("__")[-1] if "pca" not in c else c, self.transformer.get_feature_names_out()))
        return self

    def transform(self, X):
        arr = self.transformer.transform(X)
        df = pd.DataFrame(arr, columns=self.feature_names_, index=X.index)
        # Converting data to categorical type
        for c in df.columns:
            if c in self.cat_cols:
                df[c] = df[c].astype("category")
            else:
                df[c] = df[c].astype("float")
        return df

# All categorical columns
cat_cols = ["sex", 'healthcare_plan', 'procedure', 'day_of_week', 'month', 'hour']

col_transformer = DFConverter(col_transformer, cat_cols)

In [5]:
# Classifier
clf = CatBoostClassifier(cat_features=cat_cols, thread_count=-1, verbose=0)

# Search space for hyperparameters
space  = [
            Integer(10, 300, name="clf__iterations"), # Up to 300 iterations, as it is very slow to train
             Integer(1, 16, name="clf__depth"), # 16 is the maximum depth
             Real(1e-5, 1, prior="log-uniform", name="clf__learning_rate"),
             Real(1e-5, 10, prior="log-uniform", name="clf__l2_leaf_reg"),
             Real(0.0, 2.0, name="clf__bagging_temperature"),
             Real(0.1, 1.0, name="clf__rsm"),
             Integer(4, 64, name="clf__border_count"),
             Real(0.0, 1.0, name="clf__random_strength"),
             Real(1.0, 50.0, name="clf__scale_pos_weight")]

# Final pipeline, including classifier
model_pipe = Pipeline([
    ("col_transformer", col_transformer),
    ("clf", clf)
])

In [6]:
def get_train_test(data_dict, test_year, initial_year):
    # Concatenate all data from initial_year up to the year before test_year
    train_years = list(range(initial_year, test_year))
    df_train = pd.concat([data_dict[year] for year in train_years])
    df_test = data_dict[test_year]

    # Separate features and target
    X_train = df_train.drop(columns=["no_show"]).reset_index(drop=True)
    y_train = df_train["no_show"].values
    X_test = df_test.drop(columns=["no_show"])
    y_test = df_test["no_show"].values
    
    # Return weight equal to number of training folds
    return X_train, y_train, X_test, y_test, len(train_years)

class LoggingObjective:
    def __init__(self, data_per_year, test_years):
        self.data_per_year = data_per_year
        self.test_years = test_years
        self.initial_year = min(data_per_year.keys())
        self.log = []

    def evaluate_for_year(self, test_year, params):
        X_train, y_train, X_test, y_test, weight = get_train_test(
            self.data_per_year, test_year, self.initial_year
        )

        model = clone(model_pipe)
        model.set_params(**params)

        model.fit(X_train, y_train)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        score = average_precision_score(y_test, y_pred_proba)

        return score, weight

    def __call__(self, x):
        params = {dim.name: val for dim, val in zip(space, x)}

        # Parallel evaluation
        results = Parallel(n_jobs=1)(
            delayed(self.evaluate_for_year)(year, params)
            for year in self.test_years
        )
        scores, weights = zip(*results)

        # Compute weighted mean and weighted std deviation
        weighted_mean = np.average(scores, weights=weights)
        weighted_std  = np.sqrt(np.average((np.array(scores) - weighted_mean)**2, weights=weights))

        # Log full details
        self.log.append({
            'params': params,
            'scores': scores,
            'weights': weights,
            'weighted_mean': weighted_mean,
            'weighted_std': weighted_std
        })

        # gp_minimize minimizes, so return negative
        return -weighted_mean

def hyperparameter_tuning(
    data_per_year,
    test_years=[2016,2017,2018,2019,2020,2021,2022],
    log_dir="logs",
    n_calls=25,
    n_initial_points=10
):
    objective = LoggingObjective(data_per_year, test_years)

    result = gp_minimize(
        objective,
        space,
        n_calls=n_calls,
        n_initial_points=n_initial_points,
        n_jobs=1,
        verbose=True
    )

    best_score  = -result.fun
    best_params = {dim.name: val for dim, val in zip(space, result.x)}

    print(f"Best weighted average precision score: {best_score:.4f}")
    print("Best hyperparameters:")
    for k, v in best_params.items():
        print(f"  {k}: {v}")

    # Save iteration log
    os.makedirs(log_dir, exist_ok=True)
    with open(os.path.join(log_dir, "log.txt"), "w") as f:
        for i, entry in enumerate(objective.log, 1):
            f.write(f"Iteration {i}:\n")
            f.write(f"Params: {entry['params']}\n")
            f.write(f"Scores: {entry['scores']}\n")
            f.write(f"Weights: {entry['weights']}\n")
            f.write(f"Weighted Mean: {entry['weighted_mean']:.4f}\n")
            f.write(f"Weighted Std: {entry['weighted_std']:.4f}\n\n")

    return result, objective.log

In [7]:
result, objective_log = hyperparameter_tuning(data_per_year, n_calls=300, n_initial_points=100)
jb.dump(result, f"gp_minimize.pkl")
jb.dump(objective_log, f"objective_log.pkl")

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 41.8692
Function value obtained: -0.2205
Current minimum: -0.2205
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 6.4774
Function value obtained: -0.2569
Current minimum: -0.2569
Iteration No: 3 started. Evaluating function at random point.


Training has stopped (degenerate solution on iteration 9, probably too small l2-regularization, try to increase it)
Training has stopped (degenerate solution on iteration 11, probably too small l2-regularization, try to increase it)
Training has stopped (degenerate solution on iteration 9, probably too small l2-regularization, try to increase it)
Training has stopped (degenerate solution on iteration 10, probably too small l2-regularization, try to increase it)


Iteration No: 3 ended. Evaluation done at random point.
Time taken: 45.3502
Function value obtained: -0.1170
Current minimum: -0.2569
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 7.2779
Function value obtained: -0.2667
Current minimum: -0.2667
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 122.8100
Function value obtained: -0.2320
Current minimum: -0.2667
Iteration No: 6 started. Evaluating function at random point.
Iteration No: 6 ended. Evaluation done at random point.
Time taken: 159.8988
Function value obtained: -0.2457
Current minimum: -0.2667
Iteration No: 7 started. Evaluating function at random point.
Iteration No: 7 ended. Evaluation done at random point.
Time taken: 16.5696
Function value obtained: -0.1819
Current minimum: -0.2667
Iteration No: 8 started. Evaluating function at random point.
Iteration No: 8 end

Training has stopped (degenerate solution on iteration 7, probably too small l2-regularization, try to increase it)


Iteration No: 78 ended. Evaluation done at random point.
Time taken: 14.2826
Function value obtained: -0.1305
Current minimum: -0.3016
Iteration No: 79 started. Evaluating function at random point.
Iteration No: 79 ended. Evaluation done at random point.
Time taken: 9.1147
Function value obtained: -0.2832
Current minimum: -0.3016
Iteration No: 80 started. Evaluating function at random point.
Iteration No: 80 ended. Evaluation done at random point.
Time taken: 4.9940
Function value obtained: -0.1776
Current minimum: -0.3016
Iteration No: 81 started. Evaluating function at random point.
Iteration No: 81 ended. Evaluation done at random point.
Time taken: 21.8281
Function value obtained: -0.2744
Current minimum: -0.3016
Iteration No: 82 started. Evaluating function at random point.
Iteration No: 82 ended. Evaluation done at random point.
Time taken: 58.8829
Function value obtained: -0.2795
Current minimum: -0.3016
Iteration No: 83 started. Evaluating function at random point.
Iteration No

Training has stopped (degenerate solution on iteration 29, probably too small l2-regularization, try to increase it)


Iteration No: 95 ended. Evaluation done at random point.
Time taken: 100.3684
Function value obtained: -0.1524
Current minimum: -0.3016
Iteration No: 96 started. Evaluating function at random point.
Iteration No: 96 ended. Evaluation done at random point.
Time taken: 147.0374
Function value obtained: -0.1985
Current minimum: -0.3016
Iteration No: 97 started. Evaluating function at random point.
Iteration No: 97 ended. Evaluation done at random point.
Time taken: 22.1598
Function value obtained: -0.2182
Current minimum: -0.3016
Iteration No: 98 started. Evaluating function at random point.
Iteration No: 98 ended. Evaluation done at random point.
Time taken: 245.6521
Function value obtained: -0.2757
Current minimum: -0.3016
Iteration No: 99 started. Evaluating function at random point.
Iteration No: 99 ended. Evaluation done at random point.
Time taken: 357.7708
Function value obtained: -0.1859
Current minimum: -0.3016
Iteration No: 100 started. Evaluating function at random point.
Itera

['objective_log.pkl']